# Smoothing Required for a Good Coarsening $R$

In [2]:
%run /Users/olivne/helmholtz/src/helmholtz/startup.ipy

In [3]:
# Fixed seed for reproducible results.
np.random.seed(2)

# Domain size.
n = 96 # 192
domain_size = n   # With meshsize h = 1.
# Scaled wave number. Fit so lam_min = 0 (integer # periods in domain).
discretization = "5-point"
#kh = 0
kh = helmholtz.analysis.ideal.find_singular_kh(discretization, n)[0]
#kh = 0.5

repetitive = True

# Test vectors.
ideal_tv = False    # Use eigenvectors or not.
# Number of test vectors.  
num_examples = 5

# Set # sweeps manually here.
num_sweeps_on_vectors = 30 # 100 # None
threshold = 0.1

# Boottstrapping parameters.
interpolation_method = "ls" # "weighted_ls" #"ls"
fit_scheme = "ridge"
weighted = True
neighborhood = "extended" #"aggregate" # "extended"
num_test_examples = 5
leeway_factor = 1.3

_LOGGER.info("kh {}".format(kh))

INFO     kh 0.5233853805513015


## Level 0->1 Coarsening

In [4]:
# Create fine-level matrix.
a = hm.linalg.helmholtz_1d_discrete_operator(kh, discretization, n)
# Use default = Kacmzarz relaxation for kh != 0.
level0 = hm.setup.hierarchy.create_finest_level(a,  relaxer=hm.solve.relax.GsRelaxer(a) if kh == 0 else None)
# 'location' is an array of variable locations at all levels. Used only for interpolation neighbor determination.
# Finest-level variable ocations are assumed to be [0..n-1], i.e. a domain of size n with meshsize h = 1.
level0.location = np.arange(n)

### Coarsening

In [5]:
def create_coarsening(level, nu, aggregate_size, num_components):
    # Generate relaxed test vectors.
    x = hm.setup.auto_setup.get_test_matrix(level.a, nu, num_examples=num_examples)
    _LOGGER.info("Generating relaxed TF with {} sweeps, RER {:.3f}".format(nu, norm(level.a.dot(x)) / norm(x)))
    #x_log.append(x)

    # Create coarsening. Force 2 coarse vars per aggregate so we can test alignment.
    r, s = hm.repetitive.locality.create_coarsening(x, aggregate_size, num_components, normalize=False)
    R = r.tile(level.size // aggregate_size)
    return x, R

In [6]:
# Check Relaxation smoothing.
num_sweeps = hm.setup.auto_setup.check_relaxation_speed(0, level0)[1]
aggregate_size, num_components = 4, 2
x, R = create_coarsening(level0, 2 * num_sweeps, aggregate_size, num_components)

INFO     level 0 size 96 relax conv 0.93 shrinkage 0.64 PODR RER 0.55 after 5 sweeps. Work 1.0 eff 0.64
INFO     Generating relaxed TF with 10 sweeps, RER 0.312


### R Performance / Mock Cycle vs. #TF Sweeps, # Samples

In [11]:
def mock_cycle_conv(level,
                    aggregate_size: int,
                    num_components: int,
                    num_cycle_sweeps: np.ndarray,
                    num_tf_sweeps: int,
                    num_windows: int,
                    seed: int = 0,
                    weighted: bool = True):  # List[Tuple[caliber: int, r_restrict: int, symmetrize: int]] 
    np.random.seed(seed)
    # Create test functions.
    x = hm.analysis.ideal.ideal_tv(level.a, num_examples)[0] if num_tf_sweeps == -1 \
                          else hm.setup.auto_setup.get_test_matrix(level.a, num_tf_sweeps, num_examples=num_examples)
    
    # Create R, measure mock cycle rates.
    r, s = hm.repetitive.locality.create_coarsening(x, aggregate_size, num_components, normalize=False, 
                                                    num_windows=num_windows)
    R = r.tile(level.size // aggregate_size)
    return [hm.setup.auto_setup.mock_cycle_conv_factor(level, R, nu) for nu in num_cycle_sweeps]

In [12]:
seed = 0
num_cycle_sweeps = np.arange(1, num_sweeps + 4)
num_tf_sweeps = np.concatenate(([0], 2 ** np.arange(1, 7)))
num_windows = np.concatenate(([2, 3], 2 ** np.arange(2, 8)))
conv = np.array([[mock_cycle_conv(
    level0, aggregate_size, num_components, num_cycle_sweeps,
    nu, s, weighted=weighted, seed=seed)
                for s in num_windows]
                for nu in num_tf_sweeps])

Mock cycle convergnce factors below are shown as a function of
* Table = $\nu$ = number of relaxations per coarse-level correction in the mock cycle.
* Rows = $\nu_{TF}$ = number of relaxations performed on TFs.
* Columns = $s$ = number of samples (windows) used in SVD.

In [13]:
for col, nu_cycle in enumerate(num_cycle_sweeps):
    print("# Relax per cycle {}".format(nu_cycle))
    display(pd.DataFrame(conv[:, :, col], index=num_tf_sweeps, columns=num_windows))

# Relax per cycle 1


,2,3,4,8,16,32,64,128
0,0.8232,0.5194,0.5571,0.6959,0.8687,0.9159,0.9424,0.7864
2,0.6360,0.5751,0.5908,0.5909,0.5700,0.6042,0.5933,0.5924
4,0.6209,0.5933,0.5791,0.5882,0.5985,0.6000,0.5997,0.5981
8,0.6038,0.6049,0.6038,0.6024,0.5867,0.5890,0.5899,0.5887
16,0.5982,0.5996,0.6016,0.5936,0.5876,0.5854,0.5870,0.5861
32,0.5940,0.5942,0.5960,0.5874,0.5857,0.5842,0.5854,0.5848
64,0.5870,0.5875,0.5994,0.5889,0.5839,0.5840,0.5847,0.5844


# Relax per cycle 2


,2,3,4,8,16,32,64,128
0,0.7405,0.5146,0.5116,0.7228,0.8413,0.8738,0.9200,0.7953
2,0.6338,0.4040,0.3565,0.3491,0.3499,0.2938,0.3010,0.3078
4,0.3442,0.3264,0.3397,0.3428,0.3196,0.3270,0.3229,0.3194
8,0.2965,0.2884,0.2903,0.2889,0.3101,0.3028,0.3023,0.3021
16,0.3042,0.2964,0.2929,0.2991,0.3096,0.3126,0.3083,0.3102
32,0.3020,0.3019,0.3013,0.3122,0.3122,0.3196,0.3156,0.3166
64,0.3096,0.3095,0.2984,0.3077,0.3159,0.3192,0.3171,0.3183


# Relax per cycle 3


...

,2,3,4,8,16,32,64,128
0,0.6411,0.4877,0.4901,0.7052,0.8033,0.8326,0.8313,0.7557
2,0.6382,0.4051,0.4126,0.2597,0.1863,0.1390,0.1305,0.1310
4,0.3052,0.1653,0.1382,0.1592,0.1325,0.1301,0.1297,0.1299
8,0.2094,0.1926,0.1716,0.1659,0.1312,0.1319,0.1309,0.1309
16,0.1410,0.1460,0.1607,0.1391,0.1311,0.1312,0.1309,0.1310
32,0.1412,0.1418,0.1481,0.1346,0.1315,0.1315,0.1312,0.1313
64,0.1353,0.1362,0.1566,0.1369,0.1321,0.1314,0.1312,0.1313


# Relax per cycle 7


,2,3,4,8,16,32,64,128
0,0.6364,0.4911,0.4901,0.6944,0.7853,0.8124,0.8101,0.7218
2,0.6021,0.4038,0.4131,0.2679,0.1882,0.1254,0.0815,0.0821
4,0.2966,0.1613,0.1060,0.1606,0.0838,0.0846,0.0828,0.0808
8,0.2164,0.2004,0.1806,0.1718,0.0971,0.0834,0.0820,0.0827
16,0.1095,0.1455,0.1682,0.1288,0.0963,0.0843,0.0829,0.0843
32,0.1314,0.1340,0.1487,0.1105,0.0945,0.0866,0.0849,0.0857
64,0.0984,0.0999,0.1638,0.1089,0.0886,0.0859,0.0847,0.0850


# Relax per cycle 8


,2,3,4,8,16,32,64,128
0,0.6273,0.4839,0.4832,0.6847,0.7797,0.8066,0.7969,0.7189
2,0.6097,0.4025,0.4113,0.2661,0.1842,0.1250,0.0696,0.0606
4,0.3030,0.1631,0.1064,0.1589,0.0690,0.0562,0.0552,0.0551
8,0.2172,0.2012,0.1809,0.1730,0.0962,0.0687,0.0640,0.0674
16,0.1122,0.1451,0.1681,0.1300,0.0945,0.0698,0.0659,0.0712
32,0.1317,0.1347,0.1490,0.1125,0.0934,0.0771,0.0728,0.0747
64,0.1003,0.1020,0.1637,0.1115,0.0840,0.0742,0.0707,0.0715


As can be seen, even two relaxation sweps are sufficient to obtain a good coarsening, if there are enough samples; or, relaxing more (at least 8 sweeeps) with only 4 (!) windows is enough to good coarsening. In general, it seems that $R$'s quality depends on the product $\nu_{TF} s$.